In [1]:
import numpy as np
import os
import random
import matplotlib.pyplot as plt

# 常量设定
hbar = 1.0  # 普朗克常数，或者任何常数
grid_size = (64, 64, 64)  # 网格大小，可以根据需要调整
num_samples = 200  # 数据集样本数量
# 文件路径
output_dir = "data/processed/train"
velocity_dir = os.path.join(output_dir, "velocities")
abcd_dir = os.path.join(output_dir, "abcd")

# 创建文件夹
os.makedirs(velocity_dir, exist_ok=True)
os.makedirs(abcd_dir, exist_ok=True)

def generate_simple_field(size):
    """
    生成一个简单的标量场，使用单一的正弦函数。
    """
    x = np.linspace(0, 2 * np.pi, size[0])
    y = np.linspace(0, 2 * np.pi, size[1])
    z = np.linspace(0, 2 * np.pi, size[2])
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

    freq = random.uniform(0.5, 2.0)
    phase = random.uniform(0, 2 * np.pi)
    
    field = (np.sin(freq * X + phase) + np.cos(freq * Y + phase)) * np.cos(Y) * np.sin(Z)
    return field

def generate_complex_field(size):
    """
    生成一个复杂的标量场，使用多频率正弦函数叠加和随机噪声。
    """
    x = np.linspace(0, 2 * np.pi, size[0])
    y = np.linspace(0, 2 * np.pi, size[1])
    z = np.linspace(0, 2 * np.pi, size[2])
    X, Y, Z = np.meshgrid(x, y, z, indexing='ij')
    
    # 多频率正弦函数叠加
    field = (np.sin(X) * np.cos(2 * Y) * np.sin(3 * Z) +
             np.sin(2 * X) * np.cos(Y) * np.sin(Z) +
             np.sin(3 * X) * np.cos(Y) * np.sin(2 * Z))
    
    # 添加噪声
    noise_std = random.uniform(0.05, 0.2)
    noise = np.random.normal(0, noise_std, size)
    field += noise
    
    return field

def generate_random_field(size):
    """
    随机生成标量场，简单或复杂版本。
    """
    field = np.zeros(size)
    for _ in range(random.randint(1, 3)):  # 随机选择1到3个场组合
        if random.choice([True, False]):
            field += generate_simple_field(size)
        else:
            field += generate_complex_field(size)
    return field / np.max(np.abs(field))

def compute_gradient(field):
    """
    计算标量场的梯度。
    """
    grad = np.gradient(field)
    return np.array(grad)  # 返回梯度向量场

def generate_sample(hbar, grid_size):
    """
    生成一个样本，包括速度场u和物理量a, b, c, d。
    """
    a = generate_random_field(grid_size)
    b = generate_random_field(grid_size)
    c = generate_random_field(grid_size)
    d = generate_random_field(grid_size)
    
    grad_a = compute_gradient(a)
    grad_b = compute_gradient(b)
    grad_c = compute_gradient(c)
    grad_d = compute_gradient(d)
    
    u = hbar * (a * grad_b - b * grad_a + c * grad_d - d * grad_c)
    
    return u, a, b, c, d

# 可视化并保存函数
def visualize_and_save_npy_files(velocity_file, abcd_file, slice_index=16, save_dir=None, prefix="sample"):
    """
    可视化速度场和物理量的某个二维切片，并保存为图片。
    
    :param velocity_file: 速度场的 .npy 文件路径
    :param abcd_file: 物理量 (a, b, c, d) 的 .npy 文件路径
    :param slice_index: 要可视化的切片索引
    :param save_dir: 图片保存的目录。如果为 None，则不保存图片。
    :param prefix: 保存图片文件的前缀名。
    """
    # 读取 .npy 文件
    velocity = np.load(velocity_file)
    abcd = np.load(abcd_file)
    
    # 提取速度场分量
    u_x, u_y, u_z = velocity
    
    # 提取物理量 a, b, c, d
    a, b, c, d = abcd
    
    # 创建保存目录
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
    
    # 可视化速度场分量的切片
    plt.figure(figsize=(18, 6))
    
    plt.subplot(1, 3, 1)
    plt.imshow(u_x[:, :, slice_index], cmap="RdBu_r", aspect='auto')
    plt.axis('off')
    plt.subplot(1, 3, 2)
    plt.imshow(u_y[:, :, slice_index], cmap="RdBu_r", aspect='auto')
    plt.axis('off')
    plt.subplot(1, 3, 3)
    plt.imshow(u_z[:, :, slice_index], cmap="RdBu_r", aspect='auto')
    plt.axis('off')

    if save_dir:
        plt.savefig(os.path.join(save_dir, f"{prefix}_uxyz.png"), bbox_inches='tight', pad_inches=0)
    plt.close()
    
    # 可视化物理量的切片
    plt.figure(figsize=(18, 6))
    
    plt.subplot(1, 4, 1)
    plt.imshow(a[:, :, slice_index], cmap="viridis", aspect='auto')
    plt.axis('off')
    plt.subplot(1, 4, 2)
    plt.imshow(b[:, :, slice_index], cmap="viridis", aspect='auto')
    plt.axis('off')
    plt.subplot(1, 4, 3)
    plt.imshow(c[:, :, slice_index], cmap="viridis", aspect='auto')
    plt.axis('off')
    plt.subplot(1, 4, 4)
    plt.imshow(d[:, :, slice_index], cmap="viridis", aspect='auto')
    plt.axis('off')

    if save_dir:
        plt.savefig(os.path.join(save_dir, f"{prefix}_abcd.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

# 为所有训练集样本生成并保存图片
for i in range(num_samples):
    velocity_file = f'data/processed/train/velocities/sample_{i+1}_u.npy'
    abcd_file = f'data/processed/train/abcd/sample_{i+1}_abcd.npy'
    save_dir = f'data/figure/train/images/sample_{i+1}'

    visualize_and_save_npy_files(velocity_file, abcd_file, slice_index=16, save_dir=save_dir)

for i in range(num_samples_test):
    velocity_file = f'data/processed/test/velocities/sample_{i+1}_u.npy'
    abcd_file = f'data/processed/test/abcd/sample_{i+1}_abcd.npy'
    save_dir = f'data/figure/test/images/sample_{i+1}'

    visualize_and_save_npy_files(velocity_file, abcd_file, slice_index=16, save_dir=save_dir)


FileNotFoundError: [Errno 2] No such file or directory: 'data/processed/train/velocities/sample_1_u.npy'